In [2]:
!pip install -q torch langchain bitsandbytes accelerate transformers sentence-transformers faiss-gpu langchain_community gradio bcrypt pymupdf


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 MB 21.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 20.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 77.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 42.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.3/51.3 MB 34.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 89.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 420.1/420.1 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 100.4 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.2 MB/s eta 0:00:0

In [3]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import os
from dotenv import load_dotenv
import re
import os
import fitz  # PyMuPDF
import pandas as pd
import numpy as np
import gradio as gr
import bcrypt
import torch
import faiss
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from sklearn.metrics.pairwise import cosine_similarity

BASE_DIR = "/kaggle/input/ncert-dataset/dataset/12th"
UPLOAD_DIR = "/kaggle/working/uploads"
USER_DB = "/kaggle/working/user_db.csv"
INDEX_FILE = "/kaggle/working/faiss_index.index"
MODEL_PATH = "/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1"

os.makedirs(UPLOAD_DIR, exist_ok=True)

if not os.path.exists(USER_DB):
    pd.DataFrame(columns=["email", "username", "password_hash"]).to_csv(USER_DB, index=False)

embedding_model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True
)
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    quantization_config=quant_config,
    device_map="auto",
    torch_dtype=torch.float16
)
index = faiss.IndexFlatIP(768)

class SessionState:
    def __init__(self):
        self.is_authenticated = False
        self.current_user = None
        self.uploaded_files = []
        self.current_subject = None
        self.current_chapter = None
        self.current_book_path = None
        self.book_chunks = {}  # Stores (page_num, text) tuples

state = SessionState()


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [11]:
def get_subjects():
    try:
        subs = [d for d in os.listdir(BASE_DIR) if os.path.isdir(os.path.join(BASE_DIR, d))]
        return subs if subs else ["Biology", "Physics", "Chemistry"]
    except:
        return []

def get_chapters(subject):
    if not subject:
        return []
    if subject == "Uploaded Books":
        return [os.path.basename(f) for f in state.uploaded_files]
    try:
        subject_dir = os.path.join(BASE_DIR, subject)
        return [f for f in os.listdir(subject_dir) if f.endswith('.pdf')]
    except:
        return []

def process_pdf(file_path):
    try:
        doc = fitz.open(file_path)
        chunks = []
        for page_num, page in enumerate(doc):
            text = page.get_text("text").replace("\n", " ").strip()
            if len(text) > 50:
                chunks.append((page_num + 1, text))  # Store page number with text
        return chunks
    except Exception as e:
        print(f"Error processing PDF: {str(e)}")
        return []

def update_faiss_index(chunks):
    try:
        texts = [chunk[1] for chunk in chunks]
        embeddings = embedding_model.encode(texts)
        embeddings = embeddings / np.linalg.norm(embeddings, axis=1, keepdims=True)
        index.add(embeddings.astype('float32'))
        faiss.write_index(index, INDEX_FILE)
        return embeddings
    except Exception as e:
        print(f"Index update error: {str(e)}")
        return None

def generate_answer(query, context):
    try:
        prompt = f"""<s>[INST] <<SYS>>
You are an NCERT textbook expert. Answer the question precisely using ONLY the provided context.
Format your answer with clear bullet points when listing items or processes.
Use short paragraphs (1-2 sentences) for explanations.
Do NOT include any context excerpts or raw textbook content.
If the answer isn't in the context, say "This information is not available in the selected book."

Question: {query}
Context: {context[:2500]}
<</SYS>>[/INST]"""
        
        inputs = tokenizer(prompt, return_tensors="pt", max_length=2048, truncation=True).to(model.device)
        outputs = model.generate(
            **inputs,
            max_new_tokens=400,
            temperature=0.3,
            top_p=0.85,
            repetition_penalty=1.2,
            do_sample=True
        )
        
        answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
        answer = answer.split('[/INST]')[-1].strip()
        answer = answer.replace("<s>", "").replace("</s>", "")
        answer = re.sub(r'\n+', '\n', answer)
        answer = re.sub(r' +', ' ', answer)
        
        if any(keyword in query.lower() for keyword in ['list', 'steps', 'process', 'cycle']):
            answer = answer.replace('•', '\n•')
            if not answer.startswith(('•', '-', '\n•')):
                answer = '• ' + answer.replace('\n', '\n• ')
        
        return str(answer)
    
    except Exception as e:
        return f"Error generating answer: {str(e)}"



In [12]:
# Authentication functions remain same

def hash_password(password: str) -> str:
    return bcrypt.hashpw(password.encode('utf-8'), bcrypt.gensalt()).decode('utf-8')

def verify_password(stored_hash: str, provided_password: str) -> bool:
    return bcrypt.checkpw(provided_password.encode('utf-8'), stored_hash.encode('utf-8'))

def add_user(email: str, username: str, password: str) -> str:
    try:
        df = pd.read_csv(USER_DB)
        if not df[df['username'] == username].empty:
            return "Username already exists"
        new_user = pd.DataFrame([{
            'email': email,
            'username': username,
            'password_hash': hash_password(password)
        }])
        df = pd.concat([df, new_user], ignore_index=True)
        df.to_csv(USER_DB, index=False)
        return ""
    except Exception as e:
        return f"Signup error: {str(e)}"

def authenticate(username: str, password: str) -> str:
    try:
        df = pd.read_csv(USER_DB)
        user = df[df['username'] == username]
        if user.empty:
            return "User not found"
        if not verify_password(user.iloc[0]['password_hash'], password):
            return "Invalid credentials"
        return ""
    except Exception as e:
        return f"Login error: {str(e)}"


def handle_upload(files):
    if not files:
        return gr.update(), gr.update(), "No files selected", gr.update(visible=False)
    
    try:
        new_uploads = []
        for file in files:
            file_path = file.name
            filename = os.path.basename(file_path)
            save_path = os.path.join(UPLOAD_DIR, filename)
            
            with open(file_path, "rb") as f_in:
                with open(save_path, "wb") as f_out:
                    f_out.write(f_in.read())
            
            chunks = process_pdf(save_path)
            if chunks:
                update_faiss_index(chunks)
                new_uploads.append(save_path)
                state.book_chunks[save_path] = chunks
        
        state.uploaded_files.extend(new_uploads)
        return (
            gr.update(choices=["Uploaded Books"] + get_subjects()),
            gr.update(choices=get_chapters("Uploaded Books")),
            f"✅ Successfully uploaded {len(new_uploads)} file(s)",
            gr.update(visible=bool(new_uploads)))
    except Exception as e:
        return gr.update(), gr.update(), f"⚠ Error: {str(e)}", gr.update(visible=False)

def handle_query(subject, chapter, question):
    if not all([subject, chapter, question]):
        return "Please select a subject, chapter and enter your question", "", "0.0%"
    
    try:
        if subject == "Uploaded Books":
            pdf_path = next((f for f in state.uploaded_files if os.path.basename(f) == chapter), None)
        else:
            pdf_path = os.path.join(BASE_DIR, subject, chapter)
        
        if not pdf_path or not os.path.exists(pdf_path):
            return f"Selected {chapter} not found", "", "0.0%"
        
        if pdf_path != state.current_book_path:
            global index
            index = faiss.IndexFlatIP(768)
            
            if pdf_path in state.book_chunks:
                chunks = state.book_chunks[pdf_path]
            else:
                chunks = process_pdf(pdf_path)
                state.book_chunks[pdf_path] = chunks
            
            if not chunks:
                return f"No content found in {chapter}", "", "0.0%"
            
            update_faiss_index(chunks)
            state.current_book_path = pdf_path
        
        query_embed = embedding_model.encode([question])
        query_embed = query_embed / np.linalg.norm(query_embed)
        distances, indices = index.search(query_embed.astype('float32'), 5)
        
        chunks = state.book_chunks.get(pdf_path, [])
        page_numbers = set()
        
        for i in indices[0]:
            if i < len(chunks):
                page_numbers.add(chunks[i][0])
        
        if not page_numbers:
            return "No relevant information found in the selected book", "", "0.0%"
        
        context_texts = [chunks[i][1] for i in indices[0] if i < len(chunks)]
        context = "\n\n---\n\n".join(context_texts[:3])
        answer = generate_answer(question, context)
        
        page_info = f"📄 Relevant pages: {', '.join(map(str, sorted(page_numbers)))}"
        
        return answer, page_info, f"{round(float(np.mean(distances)) * 100, 1)}%"
    
    except Exception as e:
        return f"Error processing query: {str(e)}", "", "0.0%"

def reset_uploads():
    state.uploaded_files = []
    state.book_chunks = {}
    global index
    index = faiss.IndexFlatIP(768)
    state.current_book_path = None
    return [
        gr.update(choices=get_subjects()),
        gr.update(choices=[]),
        "All uploaded books have been reset",
        gr.update(visible=False)
    ]

def is_valid_email(email):
    pattern = r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'
    return re.match(pattern, email) is not None

def send_contact_email(user_email, name, subject, message):
    try:
        # SMTP Configuration (YOUR credentials for server access)
        SMTP_SERVER = "smtp.gmail.com"  # Using Gmail's SMTP
        SMTP_PORT = 587
        SMTP_USER = "panchalshruti24@gmail.com"  # Your email (for SMTP auth)
        SMTP_PASSWORD = "mffq nlno ruwy ponp"  # Google App Password
        
        # Recipient (YOU)
        RECIPIENT_EMAIL = "panchalshruti24@gmail.com"
        
        # Create email
        msg = MIMEMultipart()
        msg['From'] = f"{name} <{user_email}>"  # Shows user as sender
        msg['To'] = RECIPIENT_EMAIL  # Your email
        msg['Reply-To'] = user_email  # Replies go to user
        msg['Subject'] = f"EduQuest Contact: {subject}"
        
        # Email body
        body = f"""
        New contact form submission:
        
        Name: {name}
        Email: {user_email}
        
        Message:
        {message}
        """
        msg.attach(MIMEText(body, 'plain'))
        
        # Send using YOUR SMTP credentials
        with smtplib.SMTP(SMTP_SERVER, SMTP_PORT) as server:
            server.starttls()
            server.login(SMTP_USER, SMTP_PASSWORD)
            server.sendmail(
                from_addr=user_email,  # Actual sender (may fail without permission)
                to_addrs=RECIPIENT_EMAIL,
                msg=msg.as_string()
            )
        
        return True, "✅ Message sent successfully!"
    except Exception as e:
        return False, f"⚠ Error: {str(e)}"
        
def handle_contact_submission(name, email, subject, message):
    # Validation
    if not all([name, email, subject, message]):
        return "⚠ Please fill all required fields"
    
    if "@" not in email or "." not in email:
        return "⚠ Please enter a valid email address"
    
    # Send email
    success, status = send_contact_email(
        user_email=email,  # From form field
        name=name,
        subject=subject,
        message=message
    )
    
    return status

def update_nav_buttons(is_authenticated=False, username=""):
    return [
        gr.update(visible=not is_authenticated),
        gr.update(visible=not is_authenticated),
        gr.update(visible=is_authenticated, value=f"Hello, {username}"),
        gr.update(visible=is_authenticated),
    ]

def handle_login(u, p):
    auth_result = authenticate(u, p)
    if auth_result == "":
        state.is_authenticated = True
        state.current_user = u
        nav_updates = update_nav_buttons(True, u)
        return nav_updates + [
            gr.update(visible=False),
            gr.update(visible=False),
            gr.update(visible=False),
            gr.update(visible=True),
            gr.update(visible=False),
            gr.update(visible=False),
            gr.update(value=""),
            gr.update(value="")
        ]
    return [gr.update()] * 8 + [
        gr.update(value=f"⚠ {auth_result}"),
        gr.update()
    ]

def handle_signup(e, u, p, cp):
    if p != cp:
        return gr.update(value="⚠ Passwords do not match"), ""
    signup_result = add_user(e, u, p)
    if signup_result == "":
        return gr.update(value="✅ Account created successfully!"), ""
    return gr.update(value=f"⚠ {signup_result}"), ""

def handle_logout():
    state.is_authenticated = False
    state.current_user = None
    nav_updates = update_nav_buttons()
    return nav_updates + [
        gr.update(visible=True),
        gr.update(visible=False),
        gr.update(visible=False),
        gr.update(visible=False),
        gr.update(visible=False),
        gr.update(visible=False)
    ]

def show_home_page():
    return [
        gr.update(visible=True),
        gr.update(visible=False),
        gr.update(visible=False),
        gr.update(visible=False),
        gr.update(visible=False),
        gr.update(visible=False)
    ]

def show_about_page():
    return [
        gr.update(visible=False),
        gr.update(visible=False),
        gr.update(visible=False),
        gr.update(visible=False),
        gr.update(visible=True),
        gr.update(visible=False)
    ]

def show_contact_page():
    return [
        gr.update(visible=False),
        gr.update(visible=False),
        gr.update(visible=False),
        gr.update(visible=False),
        gr.update(visible=False),
        gr.update(visible=True)
    ]



In [ ]:

css = """
:root {
    --primary: #2563eb;
    --primary-hover: #1d4ed8;
    --secondary: #1e40af;
    --background: #f8fafc;
    --text: #1e293b;
    --light-text: #64748b;
    --border: #e2e8f0;
    --white: #ffffff;
    --danger: #dc2626;
}

body {
    font-family: 'Inter', sans-serif;
    margin: 0;
    padding: 0;
    background-color: var(--background);
}

/* Updated Navbar Styles */
.navbar {
    display: flex;
    justify-content: space-between;
    align-items: center;
    padding: 0.8rem 2rem;
    background-color: var(--white);
    box-shadow: 0 2px 10px rgba(0,0,0,0.1);
    position: sticky;
    top: 0;
    z-index: 1000;
}

.nav-buttons {
    display: flex;
    gap: 1rem;
    align-items: center;
}

.logo {
    font-size: 2rem;
    font-weight: 700;
    color: var(--primary);
    margin: 0;
    padding: 0;
    display: flex;
    align-items: center;
    gap: 0.5rem;
}

/* Updated Navigation Button Styles */
.btn-nav {
    background-color: transparent;
    color: var(--primary);
    border: none !important;
    padding: 0.6rem 1.2rem;
    font-weight: 500;
    transition: all 0.3s ease;
}

.btn-nav:hover {
    background-color: rgba(37, 99, 235, 0.05);
    transform: translateY(-2px);
    color: var(--primary-hover);
}

/* Make auth buttons match */
.auth-buttons .btn {
    border: none;
    padding: 0.6rem 1.5rem;
}

.btn-primary {
    background-color: var(--primary);
    color: white;
    box-shadow: 0 2px 4px rgba(37, 99, 235, 0.2);
}

.btn-primary:hover {
    background-color: var(--primary-hover);
    box-shadow: 0 4px 8px rgba(37, 99, 235, 0.3);
}

.btn-secondary {
    background-color: rgba(37, 99, 235, 0.1);
    color: var(--primary);
}

.btn-secondary:hover {
    background-color: rgba(37, 99, 235, 0.2);
}
/* Danger Button Styles */
.btn-danger {
    background-color: var(--danger);
    color: white;
    border: 1px solid var(--danger);
}

.btn-danger:hover {
    background-color: #b91c1c;
    box-shadow: 0 4px 12px rgba(220, 38, 38, 0.3);
}

/* Nav Link Buttons */
.nav-link-btn {
    padding: 0.6rem 1.2rem;
    font-size: 0.95rem;
    border-radius: 8px;
    transition: all 0.3s;
    text-decoration: none;
    color: var(--primary);
    background: transparent;
    border: 1px solid var(--primary);
    cursor: pointer;
    margin: 0 0.2rem;
}

.nav-link-btn:hover {
    background-color: rgba(37, 99, 235, 0.1);
    transform: translateY(-2px);
    box-shadow: 0 4px 8px rgba(0,0,0,0.1);
}

.nav-link-btn.active {
    background-color: var(--primary);
    color: white;
}

/* Auth Buttons Container */
.auth-buttons {
    display: flex;
    gap: 0.8rem;
    align-items: center;
}

/* Ripple Effect for Buttons */
.btn::after {
    content: "";
    position: absolute;
    top: 50%;
    left: 50%;
    width: 5px;
    height: 5px;
    background: rgba(255, 255, 255, 0.5);
    opacity: 0;
    border-radius: 100%;
    transform: scale(1, 1) translate(-50%);
    transform-origin: 50% 50%;
}

.btn:focus:not(:active)::after {
    animation: ripple 0.6s ease-out;
}

@keyframes ripple {
    0% {
        transform: scale(0, 0);
        opacity: 0.5;
    }
    100% {
        transform: scale(20, 20);
        opacity: 0;
    }
}

/* Rest of your existing CSS remains the same... */
.hero-section {
    display: flex;
    align-items: center;
    padding: 3rem 2rem;
    gap: 2rem;
    background-color: var(--white);
    border-radius: 12px;
    margin: 2rem;
    box-shadow: 0 4px 6px rgba(0,0,0,0.1);
}

.hero-image {
    flex: 1;
}

.hero-image img {
    width: 100%;
    border-radius: 8px;
    max-height: 400px;
    object-fit: cover;
}

.hero-text {
    flex: 1;
}

.hero-text h1 {
    font-size: 2.5rem;
    color: var(--primary);
    margin-bottom: 1rem;
}

.hero-text p {
    font-size: 1.1rem;
    color: var(--light-text);
    margin-bottom: 1.5rem;
}

.features-section {
    padding: 2rem;
    margin: 2rem;
}

.section-title {
    font-size: 1.8rem;
    color: var(--primary);
    margin-bottom: 1.5rem;
    text-align: center;
}

.feature-cards {
    display: grid;
    grid-template-columns: repeat(3, 1fr);
    gap: 2rem;
    margin-top: 2rem;
}

.feature-card {
    background-color: var(--white);
    border-radius: 12px;
    padding: 1.5rem;
    box-shadow: 0 4px 6px rgba(0,0,0,0.1);
    transition: transform 0.2s;
}

.feature-card:hover {
    transform: translateY(-5px);
}

.feature-card img {
    width: 200%;
    height: 300px;
    object-fit: cover;
    border-radius: 8px;
    margin-bottom: 1rem;
}

.feature-card h3 {
    color: var(--primary);
    margin-bottom: 0.5rem;
}

.feature-card p {
    color: var(--light-text);
}

.auth-container {
    display: flex;
    min-height: calc(100vh - 72px);
}

.auth-form {
    background-color: var(--white);
    padding: 2.5rem;
    border-radius: 12px;
    box-shadow: 0 4px 10px rgba(0,0,0,0.1);
    width: 100%;
    max-width: 400px;
    margin: 2rem auto;
}

.auth-form h2 {
    color: var(--primary);
    text-align: center;
    margin-bottom: 1.5rem;
}

.form-group {
    margin-bottom: 1.5rem;
}

.form-group label {
    display: block;
    margin-bottom: 0.5rem;
    font-weight: 500;
    color: var(--text);
}

.form-group input {
    width: 100%;
    padding: 0.75rem;
    border: 1px solid var(--border);
    border-radius: 6px;
    font-size: 1rem;
}

.form-group input:focus {
    outline: none;
    border-color: var(--primary);
    box-shadow: 0 0 0 2px rgba(37, 99, 235, 0.2);
}

.form-actions {
    margin-top: 2rem;
}

.query-container {
    padding: 2rem;
    max-width: 1200px;
    margin: 0 auto;
}

.query-header {
    display: flex;
    justify-content: space-between;
    align-items: center;
    margin-bottom: 2rem;
}

.query-title {
    font-size: 1.5rem;
    color: var(--primary);
}

.query-form {
    background-color: var(--white);
    padding: 2rem;
    border-radius: 12px;
    box-shadow: 0 4px 6px rgba(0,0,0,0.1);
}

.form-row {
    display: flex;
    gap: 1rem;
    margin-bottom: 1.5rem;
}

.form-control {
    flex: 1;
}

.form-control label {
    display: block;
    margin-bottom: 0.5rem;
    font-weight: 500;
    color: var(--text);
}

.form-control select, 
.form-control textarea {
    width: 100%;
    padding: 0.75rem;
    border: 1px solid var(--border);
    border-radius: 6px;
    font-size: 1rem;
}

.form-control textarea {
    min-height: 120px;
    resize: vertical;
}

.answer-section {
    margin-top: 2rem;
    background-color: var(--white);
    padding: 2rem;
    border-radius: 12px;
    box-shadow: 0 4px 6px rgba(0,0,0,0.1);
}

.answer-header {
    display: flex;
    justify-content: space-between;
    align-items: center;
    margin-bottom: 1rem;
}

.answer-title {
    font-size: 1.25rem;
    color: var(--primary);
}

.similarity-badge {
    background-color: var(--primary);
    color: white;
    padding: 0.25rem 0.75rem;
    border-radius: 999px;
    font-size: 0.875rem;
    font-weight: 500;
}

.answer-content {
    line-height: 1.6;
}

.context-section {
    margin-top: 1.5rem;
    background-color: #f8f9fa;
    padding: 1.5rem;
    border-radius: 8px;
}

.context-title {
    font-size: 1rem;
    color: var(--light-text);
    margin-bottom: 0.5rem;
}

.context-content {
    font-size: 0.9rem;
    line-height: 1.5;
    color: var(--text);
}

.upload-section {
    background-color: var(--white);
    padding: 2rem;
    border-radius: 12px;
    box-shadow: 0 4px 6px rgba(0,0,0,0.1);
    margin-top: 2rem;
}

.upload-title {
    font-size: 1.25rem;
    color: var(--primary);
    margin-bottom: 1.5rem;
}

.upload-actions {
    display: flex;
    gap: 1rem;
    margin-top: 1rem;
}

.upload-status {
    margin-top: 1rem;
    padding: 1rem;
    border-radius: 8px;
    background-color: #e8f5e9;
    color: #2e7d32;
}

.upload-status.error {
    background-color: #ffebee;
    color: #c62828;
}

.team-card {
    text-align: center;
    padding: 20px;
    border-radius: 10px;
    box-shadow: 0px 4px 10px rgba(0, 0, 0, 0.1);
    background: white;
    transition: transform 0.3s ease-in-out, box-shadow 0.3s ease-in-out;
}

.team-card:hover {
    transform: translateY(-5px);
    box-shadow: 0px 10px 15px rgba(0, 0, 0, 0.2);
}

.team-avatar {
    width: 120px;
    height: 120px;
    border-radius: 50%;
    margin: 0 auto 15px auto;
    object-fit: cover;
    border: 3px solid #3b8992;
}

.team-name {
    font-size: 1.2rem;
    font-weight: bold;
    margin-bottom: 5px;
}

.team-role {
    font-size: 1rem;
    color: #666;
    margin-bottom: 10px;
}

.divider {
    width: 80px;
    height: 4px;
    background-color: var(--primary);
    margin: 0.5rem auto 1.5rem auto;
}

@media (max-width: 768px) {
    .feature-cards {
        grid-template-columns: 1fr;
    }
    
    .form-row {
        flex-direction: column;
    }
    
    .navbar {
        flex-direction: column;
        padding: 1rem;
    }
    
    .nav-buttons, .auth-buttons {
        margin-top: 1rem;
        width: 100%;
        justify-content: center;
    }
}
"""
with gr.Blocks(css=css, title="EduQuest NCERT Assistant") as app:
    # Navigation Bar
    with gr.Row(elem_classes="navbar"):
        with gr.Column(scale=2, min_width=200):
            gr.Markdown("""<div class="logo">📘 EduQuest</div>""")
        with gr.Column(scale=6):
            with gr.Row(elem_classes="nav-buttons"):
                home_nav_btn = gr.Button("Home", elem_classes="btn-nav")
                about_nav_btn = gr.Button("About", elem_classes="btn-nav")
                contact_nav_btn = gr.Button("Contact", elem_classes="btn-nav")
        with gr.Column(scale=2, min_width=200):
            with gr.Row(elem_classes="auth-buttons"):
                login_nav_btn = gr.Button("Login", elem_classes="btn btn-primary", visible=True)
                signup_nav_btn = gr.Button("Sign Up", elem_classes="btn btn-primary", visible=True)
                user_greeting = gr.Markdown(visible=False)
                logout_btn = gr.Button("Logout", elem_classes="btn btn-danger", visible=False)

    # Home Page
    with gr.Column(visible=True, elem_id="home_page") as home_page:
        with gr.Row(elem_classes="hero-section"):
            with gr.Column(scale=1):
                gr.Image("https://i.pinimg.com/736x/13/b2/7b/13b27b157b72527ac7711fcff0e12985.jpg", 
                        elem_classes="hero-image",
                        show_download_button=False,
                        interactive=False)
            with gr.Column(scale=1):
                gr.Markdown("""
                <div class="hero-text">
                    <h1>The NCERT companion you've been waiting for!</h1>
                    <p>Get instant, accurate answers from NCERT books.</p>
                </div>
                """)
        
        with gr.Column(elem_classes="features-section"):
            gr.Markdown("""<div class="section-title">Features</div>""")
            with gr.Row(elem_classes="feature-cards"):
                with gr.Column():
                    gr.Markdown("""
                    <div class="feature-card">
                        <img src="https://i.pinimg.com/736x/64/f2/f6/64f2f6d9b977f7f1fa3066707856e1d2.jpg" 
                             style="pointer-events: none; user-select: none;">
                        <h3>Smart Search</h3>
                        <p>Find answers from NCERT books instantly</p>
                    </div>
                    """)
                with gr.Column():
                    gr.Markdown("""
                    <div class="feature-card">
                        <img src="https://i.pinimg.com/736x/30/b0/01/30b001306e662f8c7ef387c2bf2edb77.jpg"
                             style="pointer-events: none; user-select: none;">
                        <h3>Practice Questions</h3>
                        <p>Class-wise and subject-wise organization</p>
                    </div>
                    """)
                with gr.Column():
                    gr.Markdown("""
                    <div class="feature-card">
                        <img src="https://i.pinimg.com/736x/33/3e/a3/333ea30148468007a59f02b224f7fb1b.jpg"
                             style="pointer-events: none; user-select: none;">
                        <h3>Chapter Summary</h3>
                        <p>Upload NCERT PDFs for personalized search</p>
                    </div>
                    """)
    with gr.Column(visible=False, elem_id="login_page") as login_page:
        with gr.Column(elem_classes="auth-form"):
            gr.Markdown("""<h2>Login</h2>""")
            login_username = gr.Textbox(label="Username", placeholder="Enter your username")
            login_password = gr.Textbox(label="Password", type="password", placeholder="Enter your password")
            login_status = gr.Markdown()
            submit_login = gr.Button("Login", elem_classes="btn btn-primary")
    
    with gr.Column(visible=False, elem_id="signup_page") as signup_page:
        with gr.Column(elem_classes="auth-form"):
            gr.Markdown("""<h2>Sign Up</h2>""")
            signup_email = gr.Textbox(label="Email", placeholder="Your email address")
            signup_user = gr.Textbox(label="Username", placeholder="Choose a username")
            signup_pass = gr.Textbox(label="Password", type="password", placeholder="Create a password")
            confirm_pass = gr.Textbox(label="Confirm Password", type="password", placeholder="Confirm your password")
            signup_status = gr.Markdown()
            submit_signup = gr.Button("Sign Up", elem_classes="btn btn-primary")

    
    # Query Page
    with gr.Column(visible=False, elem_id="query_page") as query_page:
        with gr.Column(elem_classes="query-container"):
            with gr.Row(elem_classes="query-header"):
                gr.Markdown("""<div class="query-title">Query Search</div>""")
            
            with gr.Column(elem_classes="query-form"):
                with gr.Row(elem_classes="form-row"):
                    with gr.Column(elem_classes="form-control"):
                        standard_drop = gr.Dropdown(
                            ["12th", "11th", "10th", "9th", "8th"],
                            label="Select Standard",
                            value="12th"
                        )
                    with gr.Column(elem_classes="form-control"):
                        subject_drop = gr.Dropdown(
                            ["Uploaded Books"] + get_subjects(),
                            label="Select Subject",
                            interactive=True
                        )
                    with gr.Column(elem_classes="form-control"):
                        chapter_drop = gr.Dropdown(
                            [],
                            label="Select Chapter/Book",
                            interactive=True
                        )
                
                question = gr.Textbox(
                    label="Write Your Query",
                    lines=5,
                    placeholder="Type your question about the selected book/chapter..."
                )
                
                search_btn = gr.Button("Search", elem_classes="btn btn-primary")
                
                with gr.Column(elem_classes="answer-section"):
                    with gr.Row(elem_classes="answer-header"):
                        gr.Markdown("""<div class="answer-title">Answer</div>""")
                        similarity_score = gr.Markdown("""<div class="similarity-badge">0%</div>""")
                    
                    answer = gr.Markdown(elem_classes="answer-content")
                    page_display = gr.Markdown(elem_classes="context-content")
            
            with gr.Column(elem_classes="upload-section"):
                gr.Markdown("""<div class="upload-title">Upload NCERT PDFs</div>""")
                file_upload = gr.File(file_count="multiple", file_types=[".pdf"])
                
                with gr.Row(elem_classes="upload-actions"):
                    upload_btn = gr.Button("Upload", elem_classes="btn btn-primary")
                    reset_btn = gr.Button("Reset All Uploads", elem_classes="btn btn-danger", visible=False)
                
                upload_status = gr.Markdown()
 # About Us Page
    with gr.Column(visible=False, elem_id="about_page") as about_page:
        with gr.Column(elem_classes="query-container"):
            with gr.Row(elem_classes="hero-section"):
                with gr.Column(scale=1):
                    gr.Image("https://i.pinimg.com/736x/25/1d/42/251d4271955f71fef1bcaa572aca6bad.jpg", 
                           elem_classes="hero-image",interactive=False,show_download_button=False)
                with gr.Column(scale=1):
                    gr.Markdown("""
                    <div class="hero-text">
                        <h1>About EduQuest</h1>
                        <p>Empowering Students with AI-driven Learning Tools!</p>
                    </div>
                    """)
            
            with gr.Column():
                gr.Markdown("""<div class="section-title">Our Mission</div>""")
                gr.Markdown('<div class="divider"></div>', visible=False)
                gr.Markdown("""
                <div style="text-align: center;">
                    <p>At EduQuest, our mission is to revolutionize education by making learning resources more accessible, 
                    interactive, and personalized through cutting-edge AI technology. We believe that every student 
                    deserves easy access to quality educational content.</p>
                </div>
                """)
                
                gr.Markdown("""<div class="section-title">Meet the Team</div>""")
                gr.Markdown('<div class="divider"></div>', visible=False)
                
                with gr.Row():
                    with gr.Column():
                        gr.Markdown("""
                        <div class="team-card">
                            <img src="/kaggle/input/dhruvi/WhatsApp Image 2025-01-24 at 4.59.45 PM (1).jpeg" class="team-avatar">
                            <p class="team-name">Dhruvi Mahale</p>
                            <p class="team-role">22cs036@charusat.edu.in</p>
                        </div>
                        """)
                    with gr.Column():
                        gr.Markdown("""
                        <div class="team-card">
                            <img src="https://via.placeholder.com/150" class="team-avatar">
                            <p class="team-name">Shruti Panchal</p>
                            <p class="team-role">22cs044@charusat.edu.in</p>
                        </div>
                        """)
                    with gr.Column():
                        gr.Markdown("""
                        <div class="team-card">
                            <img src="https://via.placeholder.com/150" class="team-avatar">
                            <p class="team-name">Dhruvi Patel</p>
                            <p class="team-role">22cs053@charusat.edu.in</p>
                        </div>
                        """)
                
                gr.Markdown("""<div class="section-title">Why Choose EduQuest?</div>""")
                gr.Markdown('<div class="divider"></div>', visible=False)
                
                with gr.Row():
                    with gr.Column():
                        gr.Markdown("""
                        <div class="feature-card">
                            <h3>✓ AI-Powered Search</h3>
                            <p>Our advanced AI technology ensures you get the most relevant and accurate information from NCERT books instantly.</p>
                        </div>
                        """)
                    with gr.Column():
                        gr.Markdown("""
                        <div class="feature-card">
                            <h3>✓ Intuitive Interface</h3>
                            <p>Designed with students in mind, our platform is easy to navigate and user-friendly for all age groups.</p>
                        </div>
                        """)
                
                with gr.Row():
                    with gr.Column():
                        gr.Markdown("""
                        <div class="feature-card">
                            <h3>✓ Comprehensive Coverage</h3>
                            <p>Access information from all NCERT textbooks across different subjects and classes in one place.</p>
                        </div>
                        """)
                    with gr.Column():
                        gr.Markdown("""
                        <div class="feature-card">
                            <h3>✓ 24/7 Availability</h3>
                            <p>Study anytime, anywhere with our always-available platform that works on all devices.</p>
                        </div>
                        """)

    # Contact Us Page
    with gr.Column(visible=False, elem_id="contact_page") as contact_page:
        with gr.Column(elem_classes="query-container"):
            with gr.Row(elem_classes="hero-section"):
                with gr.Column(scale=1):
                    gr.Image("https://i.pinimg.com/736x/78/02/2f/78022f8934d2b67d7f005bbeb2b94a67.jpg", 
                            elem_classes="hero-image",interactive=False,show_download_button=False)
                with gr.Column(scale=1):
                    gr.Markdown("""
                    <div class="hero-text">
                        <h1>Contact Us</h1>
                        <p>We'd love to hear from you!</p>
                    </div>
                    """)
            
            with gr.Row():
                with gr.Column(scale=1):
                    gr.Markdown("""<div class="section-title">Get in Touch</div>""")
                    gr.Markdown('<div class="divider"></div>', visible=False)
                    
                    with gr.Row():
                        with gr.Column():
                            gr.Markdown("""
                            <div class="feature-card">
                                <h3>📞 Phone</h3>
                                <p>+91 9328068864</p>
                                <p>+91 8780882261</p>
                                <p>+91 8980326804</p>
                            </div>
                            """)
                        with gr.Column():
                            gr.Markdown("""
                            <div class="feature-card">
                                <h3>✉ Email</h3>
                                <p>22cs036@charusat.edu.in</p>
                                <p>22cs044@charusat.edu.in</p>
                                <p>22cs053@charusat.edu.in</p>
                            </div>
                            """)
                
                with gr.Column(scale=1):
                    gr.Markdown("""<div class="section-title">Send us a Message</div>""")
                    gr.Markdown('<div class="divider"></div>', visible=False)
                    
                    contact_name = gr.Textbox(label="Name*", placeholder="Your name")
                    contact_email = gr.Textbox(label="Email*", placeholder="Your email address")
                    contact_subject = gr.Textbox(label="Subject*", placeholder="Subject of your message")
                    contact_message = gr.Textbox(label="Message*", lines=5, placeholder="Your message here...")
                    contact_status = gr.Markdown()
                    submit_contact = gr.Button("Send Message", elem_classes="btn btn-primary")

  

    # Event Handlers
    login_nav_btn.click(
        lambda: [
            gr.update(visible=False),  # home_page
            gr.update(visible=True),   # login_page
            gr.update(visible=False),  # signup_page
            gr.update(visible=False), # query_page
            gr.update(visible=False),  # about_page
            gr.update(visible=False)   # contact_page
        ],
        outputs=[home_page, login_page, signup_page, query_page, about_page, contact_page]
    )
    
    signup_nav_btn.click(
        lambda: [
            gr.update(visible=False),  # home_page
            gr.update(visible=False),  # login_page
            gr.update(visible=True),   # signup_page
            gr.update(visible=False), # query_page
            gr.update(visible=False),  # about_page
            gr.update(visible=False)   # contact_page
        ],
        outputs=[home_page, login_page, signup_page, query_page, about_page, contact_page]
    )
    
    
    home_nav_btn.click(
        show_home_page,
        outputs=[home_page, login_page, signup_page, query_page, about_page, contact_page]
    )
    
    about_nav_btn.click(
        show_about_page,
        outputs=[home_page, login_page, signup_page, query_page, about_page, contact_page]
    )
    
    contact_nav_btn.click(
        show_contact_page,
        outputs=[home_page, login_page, signup_page, query_page, about_page, contact_page]
    )
    
    submit_login.click(
        handle_login,
        inputs=[login_username, login_password],
        outputs=[login_nav_btn, signup_nav_btn, user_greeting, logout_btn, 
                 home_page, login_page, signup_page, query_page, about_page, contact_page,
                 login_status, login_password]
    )
    
    submit_signup.click(
        handle_signup,
        inputs=[signup_email, signup_user, signup_pass, confirm_pass],
        outputs=[signup_status, confirm_pass]
    )
    
    logout_btn.click(
        handle_logout,
        outputs=[login_nav_btn, signup_nav_btn, user_greeting, logout_btn,
                 home_page, login_page, signup_page, query_page, about_page, contact_page]
    )
    
    subject_drop.change(
        lambda s: gr.update(choices=get_chapters(s)),
        inputs=subject_drop,
        outputs=chapter_drop
    )
    
    upload_btn.click(
        handle_upload,
        inputs=file_upload,
        outputs=[subject_drop, chapter_drop, upload_status, reset_btn]
    )
    
    reset_btn.click(
        reset_uploads,
        outputs=[subject_drop, chapter_drop, upload_status, reset_btn]
    )
    
    search_btn.click(
        handle_query,
        inputs=[subject_drop, chapter_drop, question],
        outputs=[answer, page_display, similarity_score]
    )

    
    submit_contact.click(
                    handle_contact_submission,
                    inputs=[contact_name, contact_email, contact_subject, contact_message],
                    outputs=contact_status
    )
    
    # Initialize navigation
    app.load(
        lambda: update_nav_buttons() + [
            gr.update(visible=False),
            gr.update(visible=False)
        ],
        outputs=[login_nav_btn, signup_nav_btn, user_greeting, logout_btn, about_page, contact_page]
    )
   
if __name__ == "__main__":
    app.launch(debug=True, share=True)

* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://ad26bf80acf6ba892b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
